<a href="https://colab.research.google.com/github/k08n/colaboratry/blob/master/cifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# RNNを利用した時系列データ予測

%matplotlib inline
import numpy as np 
import matplotlib.pyplot as plt
import math, random

import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM, GRU, SimpleRNN

In [0]:
# 時系列データの設定
noise_sigma = 0.08       # ノイズの標準偏差
steps_per_cycle = 80     # サイクルあたりのステップ数
number_of_cycles = 50    # 生成するサイクル数

# データ系列を作成
series = []
for t in range(steps_per_cycle * number_of_cycles + 1):
    d = 0.5*math.sin(t * (2*math.pi/steps_per_cycle)) + 0.5*math.cos(t*3 * (2*math.pi/steps_per_cycle)) + random.gauss(0, noise_sigma)
    series.append([d])

# 2サイクル分のグラフ表示
plt.plot(series[0:steps_per_cycle*2])    
plt.show()

In [0]:
# 入力データ, 正解データを作成
n_prev = 100      # 入力データ長
t_prev = 0        # 何サンプル先を予測するか　(0:次のサンプル)

X, Y = [], []
for i in range(len(series)-n_prev-t_prev):
    X.append(series[i:i+n_prev])        # 入力データ: i番目からi+n_prev-1番目のデータ
    Y.append(series[i+n_prev+t_prev])   # 正解データ

X = np.array(X)
Y = np.array(Y)
print(X.shape, Y.shape)

In [0]:
# 訓練データと評価用データを分割
train_ratio = 0.9           # 訓練データの割合

split_pos = int( len(X) * train_ratio )     # 分割位置
X_train = X[0:split_pos]
Y_train = Y[0:split_pos]
X_test = X[split_pos:]
Y_test = Y[split_pos:]

print('X_train shape: ', X_train.shape )
print('Y_train shape: ', Y_train.shape )
print('X_test shape: ',  X_test.shape )
print('Y_test shape: ', Y_test.shape )

In [0]:
# モデル作成
model = Sequential()  
model.add(SimpleRNN(15, input_shape=(None, 1), return_sequences=False)) 
model.add(Activation('relu'))
model.add(Dense(1, activation='linear'))  

model.summary()

In [0]:
# 学習パラメータの設定
model.compile(loss="mean_squared_error", optimizer="adam")

# モデルの学習
batch_size = 50         # バッチサイズ
epochs = 15           # エポック数
history = model.fit(X_train, Y_train,
                    batch_size=batch_size, epochs=epochs,
                    validation_data=(X_test, Y_test))

In [0]:
# 学習結果の評価
loss = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', loss)

# 学習誤差グラフ
plt.plot(history.history['loss'], label="loss")
plt.plot(history.history['val_loss'], label="val_loss")
plt.yscale('log')
plt.legend()
plt.xlabel('epoch')
plt.show()

In [0]:
# 元データと予測結果のグラフ表示
Y_predicted = model.predict(X_test)

plt.plot(Y_test[0:steps_per_cycle*2], label="Y_test")
plt.plot(Y_predicted[0:steps_per_cycle*2], label="Y_predicted")
plt.legend()
plt.show()